In [1]:
pip install -U langsmith

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langsmith import Client
from langsmith.schemas import Run, Example
from langsmith.evaluation import evaluate
import openai
from langsmith.wrappers import wrap_openai

client = Client()

# Define dataset: these are your test cases
dataset_name = "Rap Battle Dataset"
dataset = client.create_dataset(dataset_name, description="Rap battle prompts.")
client.create_examples(
    inputs=[
        {"question": "a rap battle between Atticus Finch and Cicero"},
        {"question": "a rap battle between Barbie and Oppenheimer"},
    ],
    outputs=[
        {"must_mention": ["lawyer", "justice"]},
        {"must_mention": ["plastic", "nuclear"]},
    ],
    dataset_id=dataset.id,
)

# Define AI system
openai_client = wrap_openai(openai.Client())

def predict(inputs: dict) -> dict:
    messages = [{"role": "user", "content": inputs["question"]}]
    response = openai_client.chat.completions.create(messages=messages, model="gpt-3.5-turbo")
    return {"output": response}

# Define evaluators
def must_mention(run: Run, example: Example) -> dict:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return {"key":"must_mention", "score": score}

experiment_results = evaluate(
    predict, # Your AI system
    data=dataset_name, # The data to predict and grade over
    evaluators=[must_mention], # The evaluators to score the results
    experiment_prefix="rap-generator", # A prefix for your experiment names to easily identify them
    metadata={
      "version": "1.0.0",
    },
)

View the evaluation results for experiment: 'rap-generator-019d86e4' at:
https://smith.langchain.com/o/bdfb057b-3c54-550b-8976-4c06e5176a04/datasets/5b269a5d-40d3-4ca7-8f26-d4bc7bdf24ef/compare?selectedSessions=92b4e49a-1cf6-4219-8067-5d7e471cddbf




0it [00:00, ?it/s]

: 